In [ ]:
pip install stanza


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 41.2 MB/s eta 0:00:00


In [ ]:
pip install nltk


#Task 01: Dependency Parsing with Stanza

In [ ]:
import stanza

# Download English model (run once)
stanza.download('en')

# Initialize the pipeline
nlp = stanza.Pipeline('en')

# Example sentences
sentences = [
    "The cat sees the dog.",
    "The dog chases the cat.",
    "A big dog quickly chased the small cat."
]

# Perform dependency parsing
for sentence in sentences:
    print(f"\nSentence: {sentence}")
    doc = nlp(sentence)
    for sent in doc.sentences:
        for word in sent.words:
            print(f"{word.text:<10} -> Head: {sent.words[word.head-1].text, sent.words[word.head-1].id if word.head > 0 else 'ROOT'}, Relation: {word.deprel}")


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!



Sentence: The cat sees the dog.
The        -> Head: ('cat', 2), Relation: det
cat        -> Head: ('sees', 3), Relation: nsubj
sees       -> Head: ('.', 'ROOT'), Relation: root
the        -> Head: ('dog', 5), Relation: det
dog        -> Head: ('sees', 3), Relation: obj
.          -> Head: ('sees', 3), Relation: punct

Sentence: The dog chases the cat.
The        -> Head: ('dog', 2), Relation: det
dog        -> Head: ('chases', 3), Relation: nsubj
chases     -> Head: ('.', 'ROOT'), Relation: root
the        -> Head: ('cat', 5), Relation: det
cat        -> Head: ('chases', 3), Relation: obj
.          -> Head: ('chases', 3), Relation: punct

Sentence: A big dog quickly chased the small cat.
A          -> Head: ('dog', 3), Relation: det
big        -> Head: ('dog', 3), Relation: amod
dog        -> Head: ('chased', 5), Relation: nsubj
quickly    -> Head: ('chased', 5), Relation: advmod
chased     -> Head: ('.', 'ROOT'), Relation: root
the        -> Head: ('cat', 8), Relation: det
small    

#Task 02: Constituency Parsing with NLTK

In [ ]:
import nltk
from nltk import CFG

# Define CFG
grammar = CFG.fromstring("""
  S -> NP VP
  NP -> Det N
  VP -> V NP | V
  Det -> 'the' | 'a'
  N -> 'cat' | 'dog'
  V -> 'sees' | 'chases'
""")

# Initialize Chart Parser
parser = nltk.ChartParser(grammar)

# Input sentence
sentence = ['the', 'cat', 'sees', 'the', 'dog']

# Parse and display trees
print("\nConstituency Parsing (ChartParser):")
for tree in parser.parse(sentence):
    print(tree)
    tree.pretty_print()



Constituency Parsing (ChartParser):
(S (NP (Det the) (N cat)) (VP (V sees) (NP (Det the) (N dog))))
             S              
      _______|____           
     |            VP        
     |        ____|___       
     NP      |        NP    
  ___|___    |     ___|___   
Det      N   V   Det      N 
 |       |   |    |       |  
the     cat sees the     dog



#Task 03: Top-Down Parsing with Recursive Descent Parser

In [ ]:
import nltk
from nltk import CFG

# Define CFG
grammar = CFG.fromstring("""
  S -> NP VP
  NP -> Det N
  VP -> V NP | V
  Det -> 'the' | 'a'
  N -> 'cat' | 'dog'
  V -> 'sees' | 'chases'
""")

# Initialize Recursive Descent Parser
rd_parser = nltk.RecursiveDescentParser(grammar)

# Test sentences
sentences = [
    ['the', 'cat', 'sees', 'the', 'dog'],
    ['the', 'dog', 'chases', 'the', 'cat']
]

# Parse sentences
for sent in sentences:
    print(f"\nParsing: {' '.join(sent)}")
    for tree in rd_parser.parse(sent):
        print(tree)
        tree.pretty_print()   # ASCII tree



Parsing: the cat sees the dog
(S (NP (Det the) (N cat)) (VP (V sees) (NP (Det the) (N dog))))
             S              
      _______|____           
     |            VP        
     |        ____|___       
     NP      |        NP    
  ___|___    |     ___|___   
Det      N   V   Det      N 
 |       |   |    |       |  
the     cat sees the     dog


Parsing: the dog chases the cat
(S (NP (Det the) (N dog)) (VP (V chases) (NP (Det the) (N cat))))
              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     dog chases the     cat



#Task 4: Earley Parser

#1. Initialize the chart with the start rule at position 0

In [1]:
grammar = {
    'S': [['NP', 'VP']],
    'NP': [['Det', 'N']],
    'VP': [['V', 'NP'], ['V']],
    'Det': [['the'], ['a']],
    'N': [['cat'], ['dog']],
    'V': [['sleeps'], ['sees'], ['chases']]
}

sentence = ["the", "dog", "chases", "the", "cat"]

class State:
    def __init__(self, lhs, rhs, dot, start):
        self.lhs = lhs
        self.rhs = rhs
        self.dot = dot
        self.start = start

    def is_complete(self):
        return self.dot >= len(self.rhs)

    def next_symbol(self):
        if not self.is_complete():
            return self.rhs[self.dot]
        return None

    def advance(self):
        return State(self.lhs, self.rhs, self.dot + 1, self.start)

    def __eq__(self, other):
        return (self.lhs, self.rhs, self.dot, self.start) == (other.lhs, other.rhs, other.dot, other.start)

    def __hash__(self):
        return hash((self.lhs, tuple(self.rhs), self.dot, self.start))

    def __repr__(self):
        before_dot = " ".join(self.rhs[:self.dot])
        after_dot = " ".join(self.rhs[self.dot:])
        return f"[{self.lhs} → {before_dot} • {after_dot}, {self.start}]"


# Initialize chart with start rule
chart = [set() for _ in range(len(sentence) + 1)]
start_rule = State('S', grammar['S'][0], 0, 0)
chart[0].add(start_rule)

print("Initial Chart[0]:")
for st in chart[0]:
    print("   ", st)


Initial Chart[0]:
    [S →  • NP VP, 0]


#Explanation:

We create a chart (list of sets) with one slot per word in the sentence plus one extra, and place the start rule S → • NP VP at position 0.

#2. Apply Prediction step

In [2]:
def prediction(chart, i, state, grammar):
    next_symbol = state.next_symbol()
    if next_symbol in grammar:  # Non-terminal
        for prod in grammar[next_symbol]:
            new_state = State(next_symbol, prod, 0, i)
            if new_state not in chart[i]:
                chart[i].add(new_state)
                return True
    return False


#Explanation:
If the next symbol after the dot is a non-terminal, we expand it by adding all its production rules into the chart at the current position.

#3. Apply Scanning step

In [3]:
def scanning(chart, i, state, words):
    next_symbol = state.next_symbol()
    if next_symbol is not None and i < len(words):
        if [words[i]] == [next_symbol]:  # match terminal
            new_state = state.advance()
            if new_state not in chart[i + 1]:
                chart[i + 1].add(new_state)
                return True
    return False


#Explanation:

If the next symbol after the dot is a terminal and matches the current word in the input, we move the dot forward and place the new state into the next chart position.

#4. Apply Completion step

In [4]:
def completion(chart, i, state):
    if state.is_complete():
        for prev_state in list(chart[state.start]):
            if prev_state.next_symbol() == state.lhs:
                new_state = prev_state.advance()
                if new_state not in chart[i]:
                    chart[i].add(new_state)
                    return True
    return False


#Explanation:
When a rule is fully recognized (dot at the end), we look back to the states that expected this non-terminal and advance their dots as well.

#5. Run the Earley Parser (combine steps)

In [5]:
def earley_parse(grammar, words):
    chart = [set() for _ in range(len(words) + 1)]
    start_rule = State('S', grammar['S'][0], 0, 0)
    chart[0].add(start_rule)

    for i in range(len(words) + 1):
        changed = True
        while changed:
            changed = False
            for state in list(chart[i]):
                if prediction(chart, i, state, grammar):
                    changed = True
                if scanning(chart, i, state, words):
                    changed = True
                if completion(chart, i, state):
                    changed = True

    # Print chart
    for i, states in enumerate(chart):
        print(f"\nChart[{i}]:")
        for st in states:
            print("   ", st)

    # Check acceptance
    accepting_state = State('S', grammar['S'][0], len(grammar['S'][0]), 0)
    return accepting_state in chart[len(words)]


#Explanation:
We repeatedly apply prediction, scanning, and completion at each position until no new states can be added, filling the chart.

#6. Test the parser

In [6]:
if __name__ == "__main__":
    result = earley_parse(grammar, sentence)
    print("\nAccepted?", result)



Chart[0]:
    [Det →  • a, 0]
    [NP →  • Det N, 0]
    [Det →  • the, 0]
    [S →  • NP VP, 0]

Chart[1]:
    [Det → the • , 0]
    [N →  • cat, 1]
    [N →  • dog, 1]
    [NP → Det • N, 0]

Chart[2]:
    [V →  • sees, 2]
    [N → dog • , 1]
    [VP →  • V, 2]
    [NP → Det N • , 0]
    [VP →  • V NP, 2]
    [V →  • sleeps, 2]
    [S → NP • VP, 0]
    [V →  • chases, 2]

Chart[3]:
    [VP → V • , 2]
    [Det →  • a, 3]
    [NP →  • Det N, 3]
    [V → chases • , 2]
    [VP → V • NP, 2]
    [Det →  • the, 3]
    [S → NP VP • , 0]

Chart[4]:
    [N →  • dog, 4]
    [NP → Det • N, 3]
    [Det → the • , 3]
    [N →  • cat, 4]

Chart[5]:
    [VP → V NP • , 2]
    [N → cat • , 4]
    [S → NP VP • , 0]
    [NP → Det N • , 3]

Accepted? True


#Explanation:
Finally, we check if the completed start rule (S → NP VP • , 0) is in the last chart entry, meaning the sentence is accepted by the grammar.

#Combined Summary

1.Initialize chart: Start with S → • RHS at position 0.

2.Prediction: Expand non-terminals with their productions.

3.Scanning: Match terminal words with input and advance.

4.Completion: Finish rules and update states that depend on them.

5.Iteration: Repeat steps until no new states appear in each chart column.

6.Acceptance: If the final chart contains a completed S rule, the sentence is valid.